In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import re
import numpy as np
import torch

from src.GeneratingPara.ReadMol import Molecule, check_file
from src.GeneratingPara.Generator import Generate_file
from src.GB_FFs.ParameterGenerator_ChargeTransfer_Model import OptimizedPara

GB-FFs GAFF: bond_morse=False, bool_ub=False

GB-FFs MORSE: bond_morse=True, bool_ub=False

GB-FFs UB: bond_morse=True, bool_ub=True

In [ ]:
# Prepare to process molecules 
bond_morse = False # [True, False] whether use the full Morse function for bond energy
bool_ub = False # [True, False] whether ues urey-brandley term. We use E = epsilon((r_UB / r)^2 - 1)^2 instead of harmonic fucntion
max_len = 100 # maximum number of atoms in compound, 100 is ok
print_info = False # Whether to print the details of each molecule

bool_OAT: [True, False] 

One Atom Type (OAT). In parameter file, we can either specify every atom's atom number or give the atoms under the same chemical environment the same atom number. For example, If 'bool_OAT==True', the hydrogen atoms in CH4 have the same atom number (401). Otherwise, 'H1' is 401, 'H2' is 402, 'H3' is 403, 'H4' is 404
___________

write_xyz: [True, False] 

Tinker-HP needs parameter file (.prm) and coordinate file (.xyz) to run MD. Set 'write_xyz' as 'True' will generate the coordinate file in "./GeneratingPara/xyz/" folder. 

We recommend to use the generated coordinate file to avoid the problems of atom number.
___________

waterbox: [True, False] 

If you want to simulate the system in water solution, you need set 'waterbox=True' and put the water model's (such SPC water model) parameter file as './GeneratingPara/key/waterbox.prm'.
___________

generate_waterbox: [True, False] 

If "waterbox==True" and there is no 'waterbox.prm' file in "./GeneratingPara/key/", you need use GB-FFs model to generate the 'waterbox.prm' and 'generate_waterbox' should be set to 'True'.
___________

In [ ]:
bool_OAT = True
write_xyz = True
waterbox = True
generate_waterbox = True 


'all_name' defines the molecules you want to generate parameter files.

You can either define them manually or scan the file in './GeneratingPara/sdf' folder (or './GeneratingPara/pdb' folder).

In [ ]:
all_name = []

# # define the files manually 
# all_name = ['Methanethiol', 'acetamide', 'water']

# scan the file in './GeneratingPara/sdf' folder 
for root, dirs, files in os.walk('./GeneratingPara/sdf'):
    for file in files:
        all_name.append(re.split(r'^(.+)\.sdf$',file)[1])
        
# # scan the file in './GeneratingPara/pdb' folder 
# for root, dirs, files in os.walk('./GeneratingPara/pdb'):
#     for file in files:
#         all_name.append(re.split(r'^(.+)\.pdb$',file)[1])

Fine-tuned models:

1.FineTuning-GAFF.pth: GB-FFs GAFF with GB-FFs charge (bond_morse=False, bool_ub=False)

2.SPICE-MORSE.pth: GB-FFs MORSE with GB-FFs charge (bond_morse=True, bool_ub=False)

3.SPICE-UB.pth: GB-FFs UB with GB-FFs charge (bond_morse=True, bool_ub=True)

In [ ]:
model_name = 'FineTuning-GAFF.pth' # the model to load
d_model = 512      # dimension of model (D_h)
dropout = 0.1      # dropout to eliminate overfitting
num_heads = 16     # number of attention heads in Large layers (N_heads)
nb_layer = 3       # number of Large layers (L)
activation = "SMU" # [relu,leakyrelu,gelu,SMU] the type of activation function (\sigma)

Everything is set and you just need to run the following code!!!

In [ ]:
# if you need generate a water box, the molecule 'water' must be included in 'all_name'
if generate_waterbox:
    assert 'water' in all_name
    
mol = []
for name in all_name:
    check_file(name)
    mol.append(Molecule(name, bond_morse, bool_ub, max_len, print_info=print_info))
    

In [ ]:
# Load the Fine-Tuned model
model = OptimizedPara(d_model=d_model, dropout=dropout, num_heads=num_heads, nb_layer=nb_layer, activation=activation, leakyrelu=0.1, bool_ub=bool_ub)
model.load_state_dict(torch.load('./model/' + model_name))
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
print()

If "generate_waterbox==True" the following code will generate a 'water.prm' file and move it to "./GeneratingPara/waterbox/"

In [ ]:
if generate_waterbox:
    for i in range(len(mol)):
        if mol[i].mol_name == 'water':
            Generate_file(mol[i], model, bool_OAT=True, write_xyz=False, waterbox=False)
            os.system('mv ./GeneratingPara/key/water.prm ./GeneratingPara/waterbox/waterbox.prm')
            

In [ ]:
# Generate the parameter files
for i in range(len(mol)):
    Generate_file(mol[i], model, bool_OAT=bool_OAT, write_xyz=write_xyz, waterbox=waterbox)
